**Desription**:
    
Mate Pair

**NOTE:** CrowdVar Size Range

The following notebook is a revised version of the [Crowdvariant analysis notebook](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Machine%20Learning%5BAll%20Technologies%5D.ipynb). 

In the first part of this notebook, a random forest classifier was trained on crowdsourced[CrowdVariant](https://github.com/lesleymaraina/NIST/blob/master/Notebooks/CrowdVariant_Analysis/CrowdVariantAnalysis%20-%20Data%20Collection.ipynb) data for HG002 Deletions. 

In the second part of this notebook, the trained randfom forest classifier is then used to predict labels for a new set of randomly selected HG002 Deletions.

In [1]:
"""
Imports
"""
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import LeaveOneOut
from scipy.stats import ks_2samp
from scipy import stats
from matplotlib import pyplot
from sklearn import preprocessing
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA as sklearnPCA
import plotly.plotly as py
from sklearn.cluster import DBSCAN
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import f1_score, precision_score
from sklearn import preprocessing
from ggplot import *
from bokeh.charts import TimeSeries
from bokeh.models import HoverTool
from bokeh.plotting import show
from bokeh.charts import Scatter, Histogram, output_file, show
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.palettes as palettes
from bokeh.models import HoverTool, BoxSelectTool, Legend
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning:

This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.



***
Load Data
***
Restructure features so that the training dataset and prediction dataset have all of the same features

In [2]:
### Import Data
df_crowd = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_MP.csv')
### Copy data in new dataframe as a later reference
df_crowd_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/crowd_var/CrowdVar_MP.csv')

In [3]:
df_crowd.rename(columns={'size': 'Size'}, inplace=True)

In [4]:
df_crowd.head(3)

,chrom,start,end,sample,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,...,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label,CN0_prob,CN1_prob,CN2_prob
0,1,187464828,187466479,HG002,384.891566,76.480884,83.0,8111.012048,468.334462,0.0,...,8,0.096911,0,0.0,0,0,1,0.00,0.91,0.09
1,1,33156824,33157000,HG002,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,2,0.221591,0,0.0,0,0,1,0.04,0.91,0.05
2,1,53594099,53595428,HG002,398.161765,66.303804,68.0,8229.882353,381.470742,1.0,...,3,0.059443,0,0.0,0,0,0,0.96,0.04,0.00


In [5]:
# Read in HG002 DEL dataframe
HG002_pred = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_MP.csv')
HG002_pred_2 = pd.read_csv('/Users/lmc2/NIST/Notebooks/CrowdVariant/tech_sep/svviz/svviz_MP.csv')

In [6]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(c) - set(d)

{'CN0_prob', 'CN1_prob', 'CN2_prob', 'Label'}

In [7]:
# a = ['chrom','start','end','sample','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_insertSizeScore_insertSizeScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','size','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','Label','CN0_prob','CN1_prob','CN2_prob']
# b = ['chrom','start','end','sample','id','type','SVtype','Size','Ill300x.alt_alnScore_mean','Ill300x.alt_alnScore_std','Ill300x.alt_count','Ill300x.alt_insertSize_mean','Ill300x.alt_insertSize_std','Ill300x.alt_reason_alignmentScore','Ill300x.alt_reason_insertSizeScore','Ill300x.alt_reason_orientation','Ill300x.amb_alnScore_mean','Ill300x.amb_alnScore_std','Ill300x.amb_count','Ill300x.amb_insertSize_mean','Ill300x.amb_insertSize_std','Ill300x.amb_reason_alignmentScore_alignmentScore','Ill300x.amb_reason_alignmentScore_orientation','Ill300x.amb_reason_flanking','Ill300x.amb_reason_insertSizeScore_alignmentScore','Ill300x.amb_reason_insertSizeScore_insertSizeScore','Ill300x.amb_reason_insertSizeScore_orientation','Ill300x.amb_reason_multimapping','Ill300x.amb_reason_orientation_alignmentScore','Ill300x.amb_reason_orientation_orientation','Ill300x.amb_reason_same_scores','Ill300x.ref_alnScore_mean','Ill300x.ref_alnScore_std','Ill300x.ref_count','Ill300x.ref_insertSize_mean','Ill300x.ref_insertSize_std','Ill300x.ref_reason_alignmentScore','Ill300x.ref_reason_insertSizeScore','Ill300x.ref_reason_orientation','Ill300x.GT','Ill250.alt_alnScore_mean','Ill250.alt_alnScore_std','Ill250.alt_count','Ill250.alt_insertSize_mean','Ill250.alt_insertSize_std','Ill250.alt_reason_alignmentScore','Ill250.alt_reason_insertSizeScore','Ill250.alt_reason_orientation','Ill250.amb_alnScore_mean','Ill250.amb_alnScore_std','Ill250.amb_count','Ill250.amb_insertSize_mean','Ill250.amb_insertSize_std','Ill250.amb_reason_alignmentScore_alignmentScore','Ill250.amb_reason_alignmentScore_orientation','Ill250.amb_reason_flanking','Ill250.amb_reason_insertSizeScore_alignmentScore','Ill250.amb_reason_multimapping','Ill250.amb_reason_orientation_alignmentScore','Ill250.amb_reason_orientation_orientation','Ill250.amb_reason_same_scores','Ill250.ref_alnScore_mean','Ill250.ref_alnScore_std','Ill250.ref_count','Ill250.ref_insertSize_mean','Ill250.ref_insertSize_std','Ill250.ref_reason_alignmentScore','Ill250.ref_reason_orientation','Ill250.GT','IllMP.alt_alnScore_mean','IllMP.alt_alnScore_std','IllMP.alt_count','IllMP.alt_insertSize_mean','IllMP.alt_insertSize_std','IllMP.alt_reason_alignmentScore','IllMP.alt_reason_insertSizeScore','IllMP.alt_reason_orientation','IllMP.amb_alnScore_mean','IllMP.amb_alnScore_std','IllMP.amb_count','IllMP.amb_insertSize_mean','IllMP.amb_insertSize_std','IllMP.amb_reason_alignmentScore_alignmentScore','IllMP.amb_reason_alignmentScore_orientation','IllMP.amb_reason_flanking','IllMP.amb_reason_insertSizeScore_alignmentScore','IllMP.amb_reason_insertSizeScore_insertSizeScore','IllMP.amb_reason_multimapping','IllMP.amb_reason_orientation_alignmentScore','IllMP.amb_reason_orientation_orientation','IllMP.amb_reason_same_scores','IllMP.ref_alnScore_mean','IllMP.ref_alnScore_std','IllMP.ref_count','IllMP.ref_insertSize_mean','IllMP.ref_insertSize_std','IllMP.ref_reason_alignmentScore','IllMP.ref_reason_insertSizeScore','IllMP.ref_reason_orientation','IllMP.GT','TenX.HP1_alt_alnScore_mean','TenX.HP1_alt_alnScore_std','TenX.HP1_alt_count','TenX.HP1_alt_insertSize_mean','TenX.HP1_alt_insertSize_std','TenX.HP1_alt_reason_alignmentScore','TenX.HP1_alt_reason_insertSizeScore','TenX.HP1_alt_reason_orientation','TenX.HP1_amb_alnScore_mean','TenX.HP1_amb_alnScore_std','TenX.HP1_amb_count','TenX.HP1_amb_insertSize_mean','TenX.HP1_amb_insertSize_std','TenX.HP1_amb_reason_alignmentScore_alignmentScore','TenX.HP1_amb_reason_alignmentScore_orientation','TenX.HP1_amb_reason_flanking','TenX.HP1_amb_reason_insertSizeScore_alignmentScore','TenX.HP1_amb_reason_insertSizeScore_insertSizeScore','TenX.HP1_amb_reason_multimapping','TenX.HP1_amb_reason_orientation_alignmentScore','TenX.HP1_amb_reason_orientation_orientation','TenX.HP1_amb_reason_same_scores','TenX.HP1_ref_alnScore_mean','TenX.HP1_ref_alnScore_std','TenX.HP1_ref_count','TenX.HP1_ref_insertSize_mean','TenX.HP1_ref_insertSize_std','TenX.HP1_ref_reason_alignmentScore','TenX.HP1_ref_reason_insertSizeScore','TenX.HP1_ref_reason_orientation','TenX.HP2_alt_alnScore_mean','TenX.HP2_alt_alnScore_std','TenX.HP2_alt_count','TenX.HP2_alt_insertSize_mean','TenX.HP2_alt_insertSize_std','TenX.HP2_alt_reason_alignmentScore','TenX.HP2_alt_reason_insertSizeScore','TenX.HP2_alt_reason_orientation','TenX.HP2_amb_alnScore_mean','TenX.HP2_amb_alnScore_std','TenX.HP2_amb_count','TenX.HP2_amb_insertSize_mean','TenX.HP2_amb_insertSize_std','TenX.HP2_amb_reason_alignmentScore_alignmentScore','TenX.HP2_amb_reason_alignmentScore_orientation','TenX.HP2_amb_reason_flanking','TenX.HP2_amb_reason_insertSizeScore_alignmentScore','TenX.HP2_amb_reason_insertSizeScore_insertSizeScore','TenX.HP2_amb_reason_multimapping','TenX.HP2_amb_reason_orientation_alignmentScore','TenX.HP2_amb_reason_orientation_insertSizeScore','TenX.HP2_amb_reason_orientation_orientation','TenX.HP2_amb_reason_same_scores','TenX.HP2_ref_alnScore_mean','TenX.HP2_ref_alnScore_std','TenX.HP2_ref_count','TenX.HP2_ref_insertSize_mean','TenX.HP2_ref_insertSize_std','TenX.HP2_ref_reason_alignmentScore','TenX.HP2_ref_reason_orientation','TenX.GT','pacbio.alt_alnScore_mean','pacbio.alt_alnScore_std','pacbio.alt_count','pacbio.alt_insertSize_mean','pacbio.alt_insertSize_std','pacbio.alt_reason_alignmentScore','pacbio.amb_alnScore_mean','pacbio.amb_alnScore_std','pacbio.amb_count','pacbio.amb_insertSize_mean','pacbio.amb_insertSize_std','pacbio.amb_reason_alignmentScore_alignmentScore','pacbio.amb_reason_flanking','pacbio.amb_reason_multimapping','pacbio.amb_reason_same_scores','pacbio.ref_alnScore_mean','pacbio.ref_alnScore_std','pacbio.ref_count','pacbio.ref_insertSize_mean','pacbio.ref_insertSize_std','pacbio.ref_reason_alignmentScore','pacbio.GT','GTcons','GTconflict','GTsupp','tandemrep_cnt','tandemrep_pct','segdup_cnt','segdup_pct','refN_cnt','refN_pct','GIAB_Crowd']

# set(b) - set(a)

In [8]:
### Training Set: Drop irrelevant columns and GT information
df_crowd.drop(['GTcons'], axis=1, inplace = True)
df_crowd.drop(['GTconflict'], axis=1, inplace = True)
df_crowd.drop(['GTsupp'], axis=1, inplace = True)
# df_crowd.drop('SVtype', axis=1)
# df_crowd.drop(['type'],axis=1)
df_crowd.drop(['start'],axis=1, inplace = True)
df_crowd.drop(['end'],axis=1, inplace = True)
df_crowd.drop(['chrom'],axis=1, inplace = True)
# df_crowd.drop('Size',axis=1)
df_crowd.drop(['CN0_prob'],axis=1, inplace = True)
df_crowd.drop(['CN1_prob'],axis=1, inplace = True)
df_crowd.drop(['CN2_prob'],axis=1, inplace = True)
# df_crowd.drop(['TenX.GT'],axis=1, inplace = True)
# df_crowd.drop(['pacbio.GT'],axis=1, inplace = True)
df_crowd.drop(['IllMP.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.GT'],axis=1, inplace = True)
# df_crowd.drop(['Ill300x.GT'],axis=1, inplace = True)
df_crowd.drop(['sample'],axis=1, inplace = True)
# df_crowd.columns.drop(['Label'],axis=1, inplace = True)
# df_crowd.drop(['size'],axis=1, inplace = True)
# df_crowd.drop(['Svsize'],axis=1, inplace = True)
# df_crowd.drop(['Ill250.amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)

In [9]:
# Store header names in lists and find names that are NOT contained in BOTH lists
c = list(df_crowd.columns.values)
d = list(HG002_pred.columns.values)
set(d) - set(c)

{'GIAB_Crowd',
 'GTconflict',
 'GTcons',
 'GTsupp',
 'IllMP.GT',
 'IllMP.amb_reason_insertSizeScore_alignmentScore',
 'SVtype',
 'chrom',
 'end',
 'id',
 'sample',
 'start',
 'type'}

In [10]:
### Prediction Set: Drop irrelevant columns and GT information
HG002_pred.drop(['GTcons'], axis=1, inplace = True)
HG002_pred.drop(['GTconflict'], axis=1, inplace = True)
HG002_pred.drop(['GTsupp'], axis=1, inplace = True)
HG002_pred.drop(['type'],axis=1, inplace = True)
HG002_pred.drop(['SVtype'],axis=1, inplace = True)
HG002_pred.drop(['start'],axis=1, inplace = True)
HG002_pred.drop(['end'],axis=1, inplace = True)
HG002_pred.drop(['chrom'],axis=1, inplace = True)
# HG002_pred.drop(['Size'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.GT'],axis=1, inplace = True)
# HG002_pred.drop(['pacbio.GT'],axis=1, inplace = True)
HG002_pred.drop(['IllMP.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill250.GT'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.GT'],axis=1, inplace = True)
HG002_pred.drop(['sample'],axis=1, inplace = True)
HG002_pred.drop(['id'],axis=1, inplace = True)
# HG002_pred.drop(['Ill300x.amb_reason_insertSizeScore_orientation'],axis=1, inplace = True)
HG002_pred.drop(['IllMP.amb_reason_insertSizeScore_alignmentScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP1_ref_reason_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_insertSizeScore_insertSizeScore'],axis=1, inplace = True)
# HG002_pred.drop(['TenX.HP2_amb_reason_orientation_insertSizeScore'],axis=1, inplace = True)

In [11]:
#PB Dataframe contains missing values in the label column ['GIAB_Crowd'] - will drop these rows 
HG002_pred[HG002_pred['GIAB_Crowd'].notnull()]

,Size,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,...,IllMP.ref_reason_alignmentScore,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,370.307737,...,12.0,0.0,0.0,0,0.0,1,0.679775,0,0,2
1,-90,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,356.704746,...,6.0,0.0,1.0,0,0.0,1,1.000000,0,0,2
2,-33,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,361.040645,...,20.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
3,-145,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,358.182750,...,48.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
4,-98,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,358.170900,...,27.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
5,-134,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,359.310580,...,27.0,0.0,5.0,0,0.0,0,0.000000,0,0,2
6,-22,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,357.977423,...,14.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
7,-28,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,369.415584,...,4.0,0.0,0.0,0,0.0,0,0.000000,0,0,2
8,-231,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,368.294012,...,65.0,0.0,17.0,0,0.0,0,0.000000,0,0,2
9,-172,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,375.479632,...,20.0,0.0,0.0,0,0.0,0,0.000000,0,0,2


In [12]:
df_crowd.head(3)

,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,IllMP.amb_alnScore_std,...,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,384.891566,76.480884,83.0,8111.012048,468.334462,0.0,83.0,0.0,361.339147,94.112792,...,27.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,367.782831,90.473466,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,398.161765,66.303804,68.0,8229.882353,381.470742,1.0,67.0,0.0,350.929596,97.756504,...,2.0,0.0,1329,3,0.059443,0,0.0,0,0,0


*** 
Impute missing values using KNN 
***

In [13]:
# Store data in a new variable which will be converted to a matrix
X = df_crowd
X.head(3)

,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,IllMP.amb_alnScore_std,...,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,384.891566,76.480884,83.0,8111.012048,468.334462,0.0,83.0,0.0,361.339147,94.112792,...,27.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,367.782831,90.473466,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,398.161765,66.303804,68.0,8229.882353,381.470742,1.0,67.0,0.0,350.929596,97.756504,...,2.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [14]:
# Convert dataframe to matrix
X=X.as_matrix()

#Imput missing values from three closest observations
X_imputed=KNN(k=3).complete(X)
X=pd.DataFrame(X_imputed)

Imputing row 1/1515 with 0 missing, elapsed time: 0.576
Imputing row 101/1515 with 0 missing, elapsed time: 0.625
Imputing row 201/1515 with 0 missing, elapsed time: 0.626
Imputing row 301/1515 with 0 missing, elapsed time: 0.626
Imputing row 401/1515 with 0 missing, elapsed time: 0.627
Imputing row 501/1515 with 0 missing, elapsed time: 0.627
Imputing row 601/1515 with 0 missing, elapsed time: 0.629
Imputing row 701/1515 with 0 missing, elapsed time: 0.630
Imputing row 801/1515 with 0 missing, elapsed time: 0.630
Imputing row 901/1515 with 0 missing, elapsed time: 0.630
Imputing row 1001/1515 with 0 missing, elapsed time: 0.631
Imputing row 1101/1515 with 0 missing, elapsed time: 0.631
Imputing row 1201/1515 with 0 missing, elapsed time: 0.631
Imputing row 1301/1515 with 0 missing, elapsed time: 0.632
Imputing row 1401/1515 with 0 missing, elapsed time: 0.632
Imputing row 1501/1515 with 0 missing, elapsed time: 0.632


In [15]:
# Store header values in a list, will be used later to re-label the matrix post KNN imputation
dfcrowd_header = list(df_crowd.columns.values)
X.columns = dfcrowd_header
X.head(3)

,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,IllMP.amb_alnScore_std,...,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,384.891566,76.480884,83.0,8111.012048,468.334462,0.0,83.0,0.0,361.339147,94.112792,...,27.0,0.0,1651.0,8.0,0.096911,0.0,0.0,0.0,0.0,1.0
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,367.782831,90.473466,...,0.0,0.0,176.0,2.0,0.221591,0.0,0.0,0.0,0.0,1.0
2,398.161765,66.303804,68.0,8229.882353,381.470742,1.0,67.0,0.0,350.929596,97.756504,...,2.0,0.0,1329.0,3.0,0.059443,0.0,0.0,0.0,0.0,0.0


In [16]:
# Store Labels in a new 'Y' DataFrame
Y = pd.DataFrame()
Y['Label'] = X['Label']
#Y = X.pop('Label')

In [17]:
#Count the number of labels
pd.value_counts(Y['Label'].values, sort=False)

 1.0    959
 0.0    552
-1.0      1
 2.0      3
dtype: int64

In [18]:
# Remove labels from feature set
X.drop(['Label'],axis=1, inplace = True)

In [19]:
# Order features
X4 = X.reindex_axis(sorted(X.columns), axis=1)

***
Machine Learning
***

** Train RF Classifier **

In [20]:
# Train Test Split
# Train on 70% of the data and test on 30%
X_train, X_test, y_train, y_test = train_test_split(X4, Y, test_size=0.3, random_state=0)

In [21]:
# Train Random Forest Classifier
model = RandomForestClassifier() 
model.fit(X_train, y_train)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:3: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [22]:
#NOTE: Training Set - Show number of Hom Ref, Hom Var, Het Var datapoints the model was trained on
ytrain = pd.DataFrame()
ytrain['ytrain'] = y_train
pd.value_counts(ytrain['ytrain'].values, sort=False)

 1.0    671
 0.0     73
-1.0      1
 2.0      2
dtype: int64

** Precision Score **
- Overall model performance
- Using 30% of original dataset (test set)
- Truth labels: CrowdVariant labels

In [23]:
model.predict(X_test)

array([ 0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,
        0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,
        0.,  1.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,
        0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,
        1.,  1.,  1.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  1.,  0.,  0.,
        1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,
        1.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  0.,  0

In [24]:
pred = model.predict(X_test)

In [25]:
precision_score(pred, y_test, average='micro') 

0.9516483516483516

In [26]:
# Add original labels and predicted labels back to the original dataframe
df_Xtest = pd.DataFrame(X_test)
df_Xtest.head()

,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,IllMP.amb_alnScore_std,...,IllMP.ref_reason_alignmentScore,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,Size,refN_cnt,refN_pct,segdup_cnt,segdup_pct,tandemrep_cnt,tandemrep_pct
1222,443.692308,52.919946,13.0,7277.692308,989.346123,13.0,0.0,0.0,351.255319,97.874613,...,0.0,0.0,0.0,320.0,0.0,0.0,0.0,0.000000,1.0,0.12500
310,467.500000,42.314300,8.0,6498.125000,591.488469,8.0,0.0,0.0,358.914788,94.445489,...,18.0,0.0,5.0,320.0,0.0,0.0,0.0,0.000000,1.0,0.10625
9,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,365.594086,91.226340,...,4.0,0.0,2.0,233.0,0.0,0.0,0.0,0.000000,0.0,0.00000
785,411.333333,47.590849,6.0,6700.833333,1153.827748,6.0,0.0,0.0,365.595649,93.361742,...,9.0,0.0,0.0,121.0,0.0,0.0,1.0,0.404959,1.0,0.31405
295,465.625000,40.496721,8.0,6670.500000,759.399105,8.0,0.0,0.0,364.037164,95.840659,...,12.0,0.0,0.0,324.0,0.0,0.0,0.0,0.000000,2.0,0.16358


In [27]:
labels = pd.DataFrame(y_test)

In [28]:
df_crowd.head(3)

,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,IllMP.amb_alnScore_std,...,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,Size,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,Label
0,384.891566,76.480884,83.0,8111.012048,468.334462,0.0,83.0,0.0,361.339147,94.112792,...,27.0,0.0,1651,8,0.096911,0,0.0,0,0,1
1,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,367.782831,90.473466,...,0.0,0.0,176,2,0.221591,0,0.0,0,0,1
2,398.161765,66.303804,68.0,8229.882353,381.470742,1.0,67.0,0.0,350.929596,97.756504,...,2.0,0.0,1329,3,0.059443,0,0.0,0,0,0


In [29]:
df_Xtest['predicted_label'] = pred
df_Xtest['Label'] = df_crowd_2['Label']
df_Xtest['chrom'] = df_crowd_2['chrom']
df_Xtest['start'] = df_crowd_2['start']
df_Xtest['end'] = df_crowd_2['end']
# df_Xtest['Y_test'] = labels

In [30]:
pd.value_counts(df_Xtest['Label'].values, sort=False)

0    166
1    288
2      1
dtype: int64

In [31]:
pd.value_counts(df_Xtest['predicted_label'].values, sort=False)

0.0    167
1.0    288
dtype: int64

In [32]:
from sklearn.metrics import confusion_matrix
ytest = df_Xtest['Label']
predict = df_Xtest['predicted_label']
print(confusion_matrix(ytest, predict))

[[156  10   0]
 [ 11 277   0]
 [  0   1   0]]


In [33]:
pd.crosstab(ytest, predict, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0,156,10,166
1,11,277,288
2,0,1,1
All,167,288,455


In [34]:
from sklearn.metrics import classification_report
print(classification_report(ytest, predict))

             precision    recall  f1-score   support

          0       0.93      0.94      0.94       166
          1       0.96      0.96      0.96       288
          2       0.00      0.00      0.00         1

avg / total       0.95      0.95      0.95       455



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



***
Predict
***

**Load Data**

In [35]:
HG002_pred.head(3)

,Size,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,...,IllMP.ref_reason_alignmentScore,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.307737,...,12.0,0.0,0.0,0,0.0,1,0.679775,0,0,2
1,-90,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356.704746,...,6.0,0.0,1.0,0,0.0,1,1.000000,0,0,2
2,-33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.040645,...,20.0,0.0,0.0,0,0.0,0,0.000000,0,0,2


In [36]:
X2 = HG002_pred

** Impute missing values using KNN **

In [37]:
#Convert dataframe to matrix
X2=X2.as_matrix()

#Imput missing values from three closest observations
X2_imputed=KNN(k=3).complete(X2)
X2=pd.DataFrame(X2_imputed)

Imputing row 1/2805 with 0 missing, elapsed time: 2.008
Imputing row 101/2805 with 0 missing, elapsed time: 2.010
Imputing row 201/2805 with 0 missing, elapsed time: 2.010
Imputing row 301/2805 with 0 missing, elapsed time: 2.011
Imputing row 401/2805 with 0 missing, elapsed time: 2.012
Imputing row 501/2805 with 0 missing, elapsed time: 2.013
Imputing row 601/2805 with 0 missing, elapsed time: 2.015
Imputing row 701/2805 with 0 missing, elapsed time: 2.016
Imputing row 801/2805 with 0 missing, elapsed time: 2.018
Imputing row 901/2805 with 0 missing, elapsed time: 2.020
Imputing row 1001/2805 with 0 missing, elapsed time: 2.023
Imputing row 1101/2805 with 0 missing, elapsed time: 2.024
Imputing row 1201/2805 with 0 missing, elapsed time: 2.026
Imputing row 1301/2805 with 0 missing, elapsed time: 2.028
Imputing row 1401/2805 with 0 missing, elapsed time: 2.030
Imputing row 1501/2805 with 0 missing, elapsed time: 2.031
Imputing row 1601/2805 with 0 missing, elapsed time: 2.033
Imputing 

In [38]:
dfcrowd_header = list(HG002_pred.columns.values)
X2.columns = dfcrowd_header
X2.head(3)

,Size,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,...,IllMP.ref_reason_alignmentScore,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.307737,...,12.0,0.0,0.0,0.0,0.0,1.0,0.679775,0.0,0.0,2.0
1,-90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356.704746,...,6.0,0.0,1.0,0.0,0.0,1.0,1.000000,0.0,0.0,2.0
2,-33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.040645,...,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0


In [39]:
X3 = pd.DataFrame()
X3 = X2
X3.head(3)

,Size,IllMP.alt_alnScore_mean,IllMP.alt_alnScore_std,IllMP.alt_count,IllMP.alt_insertSize_mean,IllMP.alt_insertSize_std,IllMP.alt_reason_alignmentScore,IllMP.alt_reason_insertSizeScore,IllMP.alt_reason_orientation,IllMP.amb_alnScore_mean,...,IllMP.ref_reason_alignmentScore,IllMP.ref_reason_insertSizeScore,IllMP.ref_reason_orientation,tandemrep_cnt,tandemrep_pct,segdup_cnt,segdup_pct,refN_cnt,refN_pct,GIAB_Crowd
0,-178.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,370.307737,...,12.0,0.0,0.0,0.0,0.0,1.0,0.679775,0.0,0.0,2.0
1,-90.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,356.704746,...,6.0,0.0,1.0,0.0,0.0,1.0,1.000000,0.0,0.0,2.0
2,-33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,361.040645,...,20.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,2.0


In [40]:
X2.drop(['GIAB_Crowd'],axis=1, inplace = True)

In [41]:
# Order features
X5 = X2.reindex_axis(sorted(X2.columns), axis=1)

In [42]:
model.predict(X5)

array([ 0.,  1.,  1., ...,  0.,  0.,  0.])

In [43]:
df_new = pd.DataFrame()
df_new['true'] = HG002_pred['GIAB_Crowd'].astype(float)
df_new['predicted'] = model.predict(X5)
df_new['predicted'] = df_new['predicted'].astype(float)

In [44]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     473
1.0    1734
0.0     598
dtype: int64

In [45]:
pd.value_counts(df_new['predicted'].values, sort=False)

0.0     849
1.0    1956
dtype: int64

In [46]:
# Step for processing missing values in label column
df_new_ = pd.DataFrame()
df_new_ = df_new[df_new['true'].notnull()]
df_new_

,true,predicted
0,2.0,0.0
1,2.0,1.0
2,2.0,1.0
3,2.0,1.0
4,2.0,1.0
5,2.0,1.0
6,2.0,1.0
7,2.0,1.0
8,2.0,1.0
9,2.0,1.0


In [47]:
df_new['true']

0       2.0
1       2.0
2       2.0
3       2.0
4       2.0
5       2.0
6       2.0
7       2.0
8       2.0
9       2.0
10      2.0
11      2.0
12      2.0
13      2.0
14      2.0
15      2.0
16      2.0
17      2.0
18      2.0
19      2.0
20      2.0
21      2.0
22      2.0
23      2.0
24      2.0
25      2.0
26      2.0
27      2.0
28      2.0
29      2.0
       ... 
2775    0.0
2776    0.0
2777    0.0
2778    0.0
2779    0.0
2780    0.0
2781    0.0
2782    0.0
2783    0.0
2784    0.0
2785    0.0
2786    0.0
2787    0.0
2788    0.0
2789    0.0
2790    0.0
2791    0.0
2792    0.0
2793    0.0
2794    0.0
2795    0.0
2796    0.0
2797    0.0
2798    0.0
2799    0.0
2800    0.0
2801    0.0
2802    0.0
2803    0.0
2804    0.0
Name: true, dtype: float64

In [48]:
# Calculate Precision Score
true = df_new_['true']
predicted = df_new_['predicted']
precision_score(true, predicted, average='micro') 

0.73725490196078436

In [49]:
pd.value_counts(df_new['true'].values, sort=False)

2.0     473
1.0    1734
0.0     598
dtype: int64

In [50]:
pd.crosstab(true, predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0.0,1.0,All
True,,,
0.0,536,62,598
1.0,202,1532,1734
2.0,111,362,473
All,849,1956,2805


In [51]:
from sklearn.metrics import classification_report
true = df_new_['true']
pred = df_new_['predicted']
print(classification_report(true, pred))

             precision    recall  f1-score   support

        0.0       0.63      0.90      0.74       598
        1.0       0.78      0.88      0.83      1734
        2.0       0.00      0.00      0.00       473

avg / total       0.62      0.74      0.67      2805



/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [52]:
# # Store Chr X and Y as numbers (23, 24)
# X5['chrom'].replace('X', 23, inplace=True)
# X5['chrom'].replace('Y', 24, inplace=True)

In [53]:
pred_prob = model.predict_proba(X5)

In [54]:
X5['predicted_label'] = pred
X5['Label'] = HG002_pred_2['GIAB_Crowd']
X5['chrom'] = HG002_pred_2['chrom']
X5['start'] = HG002_pred_2['start']
X5['end'] = HG002_pred_2['end']

In [59]:
X6 = pd.concat([X5, pd.DataFrame(pred_prob, columns=['1','2','3','4'])])

In [60]:
X6.to_csv('MP_pred_prob.csv', index=False)

In [61]:
X6 = pd.read_csv('MP_pred_prob.csv')

In [62]:
X6.rename(columns={'1': 'unknown'}, inplace=True)
X6.rename(columns={'2': 'Hom_Var'}, inplace=True)
X6.rename(columns={'3': 'Het_Var'}, inplace=True)
X6.rename(columns={'4': 'Hom_Ref'}, inplace=True)

** Show Size Distribution of Labels with Predict Probability > 0.8 **

In [63]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)

In [64]:
X_homVar_conf = X6[(X6['Hom_Var'] == 0.9) | (X6['Hom_Var'] == 1) | (X6['Hom_Var'] == 0.8)]

In [65]:
X_homVar_conf['Size'] = HG002_pred_2['Size'].abs()

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [66]:
output_notebook()

Loading BokehJS ...

** Homozygous **

In [67]:
# all_df_del['abs_size'] = all_df_del['Size'].abs()
X_homVar_conf['Size_Log'] = np.log10(X_homVar_conf.Size)

/Users/lmc2/anaconda/envs/NIHFAES/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [68]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_conf, values='Size_Log', title='Homozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(Log10)", ylabel="Frequency")
output_file("300x_HomVar_PredProb0.8+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/300x.Hom.0.8%2B.png)

** Heterozygous **

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_conf = X6[(X6['Het_Var'] == 0.9) | (X6['Het_Var'] == 1) | (X6['Het_Var'] == 0.8)]
X_hetVar_conf['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_conf['Size_Log'] = np.log10(X_hetVar_conf.Size)

In [ ]:
output_notebook()

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_conf, values='Size_Log', title='Heterozygous Variant: Predict Proba > 0.8', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("300x_HetVar_PredProb0.8+.html")
show(p)

![Figure 2](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/300x.Het.0.8%2B.png)

** Show Size Distribution of Labels with Predict Probability < 0.3 **

In [ ]:
X6['Hom_Var_str'] = X6['Hom_Var'].astype(str)
X_homVar_least = X6[(X6['Hom_Var'] == 0.3) | (X6['Hom_Var'] == 0.2) | (X6['Hom_Var'] == 0.1)]
X_homVar_least['Size'] = HG002_pred_2['Size'].abs()
X_homVar_least['Size_Log'] = np.log10(X_homVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_homVar_least, values='Size_Log', title='Homozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("300x_HomVar_PredProb0.3-.html")
show(p)

![Figure 1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/300x.Hom.0.3-.png)

** Heterozygous **

In [ ]:
X6['Het_Var_str'] = X6['Het_Var'].astype(str)
X_hetVar_least = X6[(X6['Het_Var'] == 0.3) | (X6['Het_Var'] == 0.2) | (X6['Het_Var'] == 0.1)]
X_hetVar_least['Size'] = HG002_pred_2['Size'].abs()
X_hetVar_least['Size_Log'] = np.log10(X_hetVar_least.Size)

In [ ]:
# Plot the counts of each predicted probability for Het Var events
p = figure()
p = Histogram(X_hetVar_least, values='Size_Log', title='Heterozygous Variant: Predict Proba < 0.3', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("300x_HetVar_PredProb0.3-.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/CrowdVariant_Analysis/Images/300x.Het.0.3-.png)

** Size Distribution of Original Data **

In [ ]:
p = figure()
p = Histogram(df_crowd_2, values='size', title='CrowdVariant Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(bp)", ylabel="Frequency")
output_file("CrowdVar.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/crowdVar_size.png)

In [ ]:
del_data = pd.read_csv('/Users/lmc2/Desktop/log_Size_.csv')

In [ ]:
p = figure()
p = Histogram(del_data, values='DEL_log_size', title='SVVIZ HG002 Rand Selection Data Size Distribution', color='LightSlateGray', bins=30, xlabel="Size(log10)", ylabel="Frequency")
output_file("SVVIZ.Size.html")
show(p)

![Figure1](https://raw.githubusercontent.com/lesleymaraina/NIST/master/Notebooks/svviz_size.png)

**TODO**

[Feature Importance](http://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html)